In [19]:
import networkx as nx
import csv
import time
from collections import defaultdict
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import random


In [20]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import random

# === Configuration ===
TIMETABLE_FILE = "C:/Users/srupe/Downloads/crew/mainLoop_rupesh.csv"
MIN_RAKE_GAP_MINUTES = 30   # Minimum required gap between different rakes
RANDOM_SAMPLE_SIZE = 300    # Number of services to select
SEED = 42                  # Seed for reproducibility
ALLOWED_RAKE_CHANGE_STATIONS = {"KKDA", "PVGW"}  # Allowed rake-change stations
MAX_CONNECTION_GAP_MINUTES = 120   

# === Utility Functions ===
def parse_time_to_minutes(t):
    """Converts a time string 'HH:MM' into minutes since midnight."""
    try:
        t = t.strip()
        hours, minutes = map(int, t.split(":"))
        return hours * 60 + minutes
    except:
        return None

def get_base_station_name(station):
    """Extracts the base station name (first word) in uppercase."""
    if pd.isna(station):
        return None
    return station.strip().split()[0].upper()





def rake_feasible_connection(end_station, start_station, end_time, start_time, rake_a, rake_b):
    """
    A -> B is feasible iff:
      - Same base station
      - 0 <= (start - end) < 120
      - If rake changes: station must be in ALLOWED_RAKE_CHANGE_STATIONS and gap >= MIN_RAKE_GAP_MINUTES
    """
    if end_station != start_station:
        return False

    # # Guard against missing times
    # if end_time is None or start_time is None:
    #     return False

    gap = start_time - end_time

    # Next service must start after (or exactly when) the previous ends, and within 120 mins
    if gap < 0 or gap > MAX_CONNECTION_GAP_MINUTES:
        return False

    if rake_a == rake_b:
        # Same rake: gap already checked (0 <= gap <= 120)
        return True
    else:
        # Rake change: only at allowed stations and must satisfy the min gap too
        return (end_station in ALLOWED_RAKE_CHANGE_STATIONS) and (gap >= MIN_RAKE_GAP_MINUTES)





# === Load and Process Data ===
df = pd.read_csv(TIMETABLE_FILE)
df.columns = df.columns.str.strip()
df['Service'] = df['Service'].astype(str)

# Convert time columns to minutes
df['Start Minutes'] = df['Start Time'].apply(parse_time_to_minutes)
df['End Minutes'] = df['End Time'].apply(parse_time_to_minutes)

# Extract base station names
df['Base Start Station'] = df['Start Station'].apply(get_base_station_name)
df['Base End Station'] = df['End Station'].apply(get_base_station_name)

# === Randomly Sample Services ===
random.seed(SEED)
selected_services = random.sample(df['Service'].unique().tolist(), k=RANDOM_SAMPLE_SIZE)
df_sampled = df[df['Service'].isin(selected_services)].copy()
df_sampled.reset_index(drop=True, inplace=True)

# === Build Graph ===
G = nx.DiGraph()
services = df_sampled['Service'].tolist()

# Add service nodes plus source and sink
G.add_nodes_from(services + ["S", "T"])

# Add edges from source to each service
for service in services:
    G.add_edge("S", service, color="black")  # source edges in black

# Add feasible service-to-service connections
for i, row_a in df_sampled.iterrows():
    service_a = row_a['Service']
    end_station = row_a['Base End Station']
    end_time = row_a['End Minutes']
    rake_a = row_a.get('Rake Num', None)

    for j, row_b in df_sampled.iterrows():
        service_b = row_b['Service']
        if service_a == service_b:
            continue

        start_station = row_b['Base Start Station']
        start_time = row_b['Start Minutes']
        rake_b = row_b.get('Rake Num', None)

        if rake_feasible_connection(end_station, start_station, end_time, start_time, rake_a, rake_b):
            # Color edges: red if rake change, gray if same rake
            edge_color = "red" if rake_a != rake_b else "gray"
            G.add_edge(service_a, service_b, color=edge_color)

# Add edges from each service to sink
for service in services:
    G.add_edge(service, "T", color="black")  # sink edges in black

# # === Visualize the Graph ===
# plt.figure(figsize=(60, 45))
# pos = nx.spring_layout(G, seed=42, k=0.3, iterations=100)

# # Extract edge colors
# edge_colors = [G[u][v]['color'] for u, v in G.edges()]

# nx.draw(
#     G,
#     pos,
#     with_labels=True,
#     node_color='skyblue',
#     node_size=9000,
#     font_size=8,
#     font_weight='bold',
#     edge_color=edge_colors,
#     width=1.3,
#     arrows=True,
#     arrowsize=10,
#     connectionstyle="arc3,rad=0.2"
# )

# plt.title(f"Random {RANDOM_SAMPLE_SIZE} Services: Feasibility Graph (Station + Time + Rake, Rake Change at KKDA/PVGW Only)", fontsize=18)
# plt.axis("off")
# plt.tight_layout()
# plt.show()


In [21]:
print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())


print("\nNodes:")
print(list(G.nodes()))

print("\nEdges:")
print(list(G.edges()))


Number of nodes: 302
Number of edges: 3481

Nodes:
['727', '10', '11', '13', '721', '379', '17', '380', '20', '733', '22', '734', '735', '23', '24', '385', '571', '739', '33', '573', '35', '574', '388', '389', '39', '576', '41', '577', '42', '43', '578', '392', '46', '47', '580', '51', '582', '583', '396', '54', '55', '584', '56', '58', '398', '59', '61', '400', '63', '65', '66', '590', '70', '71', '406', '74', '77', '594', '410', '595', '412', '85', '414', '88', '598', '89', '599', '415', '91', '600', '416', '92', '95', '603', '102', '422', '106', '109', '110', '612', '115', '116', '613', '428', '117', '429', '615', '119', '430', '122', '431', '124', '125', '432', '128', '129', '433', '621', '130', '434', '622', '133', '134', '623', '436', '136', '142', '627', '143', '144', '628', '145', '146', '147', '149', '444', '152', '154', '634', '156', '158', '163', '166', '168', '167', '172', '457', '175', '645', '179', '180', '461', '462', '187', '188', '190', '191', '192', '652', '468', '196

In [22]:
# Pick a node of interest
node_of_interest = "656"   # replace with your service ID or "S" / "T"

# Get successors and predecessors
succ = list(G.successors(node_of_interest))
pred = list(G.predecessors(node_of_interest))

print(f"Node: {node_of_interest}")
print(f"Successors ({len(succ)}): {succ}")
print(f"Predecessors ({len(pred)}): {pred}")


Node: 656
Successors (11): ['667', '225', '669', '670', '235', '674', '240', '676', '245', '249', 'T']
Predecessors (11): ['S', '143', '145', '147', '152', '154', '634', '156', '163', '167', '180']


In [23]:
# === Jurisdiction Buckets ===
jurisdiction_dict = {
    1: {'MKPR','MKPR UP','MKPR DN','SAKP','DDSC','DDSC DN PF','DDSC SDG','DDSC SDG STABLE (DAY)','DDSC DN',
        'DDSC SDG','PVGW','PVGW UP','PVGW DN','MKPD','MKPD','SAKP 3RD','SAKP 3RD PF','MKPR DN SDG','MKPR DN PF','DDSC DN SDG'},
    2: {'MUPR DN SDG STABLE (DAY)','MUPR 4TH SDG STABLE (DAY)','MUPR 3RD SDG STABLE','SVVR','SVVR DN','MUPR',
        'MUPR DN','MUPR 4TH','MUPR 3RD SDG','KKDA DN','KKDA UP','IPE','IPE 3RD PF','IPE 3RD','VND','VND (M)',
        'MVPO','MVPO DN','NZM','NIZM','KKDA','MUPR DN SDG','MVPO DN PF','SVVR DN PF','MUPR 3RD SDG','MUPR 4TH PF',
        'MUPR 4TH SDG','MUPR DN PF','MUPR DN SDG','MUPR DN SDG'}
}

# === Lookup dictionaries ===
start_time_dict = pd.Series(df['Start Minutes'].values, index=df['Service']).to_dict()
end_time_dict = pd.Series(df['End Minutes'].values, index=df['Service']).to_dict()
start_station_dict = pd.Series(df['Base Start Station'].values, index=df['Service']).to_dict()
end_station_dict = pd.Series(df['Base End Station'].values, index=df['Service']).to_dict()
service_time_dict = pd.Series(df['service time'].values, index=df['Service']).to_dict()

## Checking Current Path

In [24]:
def is_path_acceptable(
    path, end,
    total_driving_time, driving_time_limit,
    duty_time_limit
):
    # def get_jurisdiction_groups(station):
    #     return {
    #         Jurisdiction_group_id for Jurisdiction_group_id, stations in jurisdiction_dict.items()
    #         if station in stations
    #     }

    # -----------------------------
    # Condition 1: Driving time limit
    # -----------------------------
    if total_driving_time > driving_time_limit:
        return False

    # -----------------------------
    # Condition 2: Duty time limit
    # -----------------------------
    first_service_start_time = start_time_dict.get(path[1])

    if path[-1] == end:
        last_service = path[-2]
    else:
        last_service = path[-1]

    last_service_end_time = end_time_dict.get(last_service)
    total_duty_time = last_service_end_time - first_service_start_time

    if total_duty_time > duty_time_limit:
        return False

    # -----------------------------
    # Condition : Jurisdiction match (only at end)
    # -----------------------------
    # if neighbor == end:
    #     start_station_first_duty = start_station_dict.get(path[1])
    #     end_station_last_duty = end_station_dict.get(path[-2])

    #     start_groups = get_jurisdiction_groups(start_station_first_duty)
    #     end_groups = get_jurisdiction_groups(end_station_last_duty)

    #     if start_groups.isdisjoint(end_groups):
    #         return False

    # -----------------------------
    # Condition 3: Continuous driving 180-min rule
    # -----------------------------
    continuous_drive = 0
    for i in range(1, len(path)-1):  # skip 'T' at the end
        current_service = path[i]
        next_service = path[i+1]

        # Skip dummy nodes
        if current_service not in service_time_dict or next_service not in start_time_dict:
            continue

        service_time = service_time_dict.get(current_service, 0)
        continuous_drive += service_time

        # Gap to next service
        gap = start_time_dict[next_service] - end_time_dict[current_service]

        # Check if continuous block exceeds 180
        if continuous_drive > 180:
            return False  # cannot exceed 180

        # If exactly 180, next gap must be >=50
        if continuous_drive == 180:
            if gap < 50:
                return False
            else:
                continuous_drive = 0  # reset after required break

        # If gap >30 (normal break), reset continuous driving counter
        elif gap > 30:
            continuous_drive = 0

    # -----------------------------
    # All conditions passed
    # -----------------------------
    return True


## Checking Final Path

In [25]:
def is_final_path_valid(path):
    """
    Check if a completed path ending at 'T' is valid based on:
    1. Jurisdiction overlap between first and last duty.
    2. Required break conditions (30min + 50min OR only >=50min).
    """

    # === Jurisdiction check ===
    start_station_first_duty = start_station_dict.get(path[1])
    end_station_last_duty = end_station_dict.get(path[-2])

    start_groups = get_jurisdiction_groups(start_station_first_duty)
    end_groups = get_jurisdiction_groups(end_station_last_duty)

    # If no overlap in jurisdiction groups = invalid path
    if start_groups.isdisjoint(end_groups):
        return False

    # === Break check ===
    if not has_required_breaks(path):
        return False

    return True


def get_jurisdiction_groups(station):
    """
    Return the set of jurisdiction groups a station belongs to.
    """
    return {
        Jurisdiction_group_id
        for Jurisdiction_group_id, stations in jurisdiction_dict.items()
        if station in stations
    }


def has_required_breaks(path):
    """
    Check if the path has the required breaks:
    Case 1: At least one 30min break AND one 50min break
    Case 2: At least one break >=50min (even if no 30min break exists)
    """
    if len(path) < 6:
        return False  # skipping short paths

    start_times = [start_time_dict[s] for s in path[1:-1]]
    end_times = [end_time_dict[s] for s in path[1:-1]]

    gaps = [
        start_times[i + 1] - end_times[i]
        for i in range(len(start_times) - 1)
    ]

    break_stations = {"KKDA", "PVGW"}  # Breaks allowed only here

    has_30min_break = False
    has_50min_break = False

    for i, gap in enumerate(gaps):
        # station where this break happens = end of current service
        station = end_station_dict.get(path[1 + i])

        # if station in break_stations:
        #     if gap >= 30:
        #         has_30min_break = True
        #     if gap >= 50:
        #         has_50min_break = True



        if station in break_stations:
            if 30 <= gap < 50:   # lower + upper bound
                has_30min_break = True
            if 50 <= gap < 120:   # lower + upper bound
                has_50min_break = True           

    return (has_30min_break and has_50min_break) or has_50min_break


## Stack

In [26]:
def stack_based_all_paths(
    G, start='S', end='T',
    driving_time_limit=360, duty_time_limit=445,
    
):
    stack = [(start, [start], 0)]  # (current_node, path_so_far, total_driving_time)
    n = 0

    while stack:
        current_node, path, total_driving_time = stack.pop()

        for neighbor in G.successors(current_node):
            service_time = service_time_dict.get(neighbor, 0)
            new_total_driving_time = total_driving_time + service_time
            new_path = path + [neighbor]

            if is_path_acceptable(
                new_path, end,
                new_total_driving_time, driving_time_limit,
                duty_time_limit
            ):
                if neighbor == end:
                    if is_final_path_valid(new_path):
                        n += 1
                       # print(new_path)
                        
                else:
                    stack.append((neighbor, new_path, new_total_driving_time))

    print(f"Total valid paths found: {n}")


In [27]:
start_time = time.time()

print(f"Number of services in the graph {RANDOM_SAMPLE_SIZE}")
stack_based_all_paths(
    G, start='S', end='T', driving_time_limit=360, duty_time_limit=445
) 

end_time = time.time()

print(f"Execution time: {end_time - start_time:.4f} seconds")

Number of services in the graph 300
Total valid paths found: 152893
Execution time: 2.6150 seconds
